In [57]:
import sys
sys.path.append("../ARCH_package")

import damage, plot

import dill
import pandas as pd
import os

In [58]:
# Create path for exporting
path = '../Results/Summary tables/'
if not os.path.exists(path):
    os.makedirs(path)

In [59]:
with open('../Exports/lbc_model_comparison.dill', 'rb') as infile:
    lbc_list = dill.load(infile)


# Update trajectory type in lbc cohort
fit_mutation_ids = [mutation.id for mutation in lbc_list
                    if mutation.optimal_model =='fit mutation']

# LiFT summary

In [60]:
# Import non-synonymous mutations as exported in LiFT.py
with open('../Exports/LBC_non-synonymous_LiFT_fitted.dill', 'rb') as infile:
    lbd_LiFT = dill.load(infile)


In [61]:
# Import non-synonymous mutations as exported in LiFT.py
with open('../Exports/LBC_non-synonymous_LiFT_fitted.dill', 'rb') as infile:
    lbc_LiFT = dill.load(infile)


# Outlier p_value concordance
fig, outlier_pvalue_mutation_list = (
    plot.outlier_pvalue_comparison(lbc_LiFT, fit_mutation_ids))

for part in lbc_LiFT:
    for traj in part.trajectories:
        traj.outlier_pvalue = None
        if traj.mutation in [mutation.id for mutation in outlier_pvalue_mutation_list]:
            pvalue = [mutation.outlier_pvalue for mutation in outlier_pvalue_mutation_list if mutation.id == traj.mutation][0]
            traj.outlier_pvalue = pvalue

damage_df = damage.damage

# Update damage class attribute for each trajectory in the cohort
for part in lbc_LiFT:
    for traj in part.trajectories:
        traj.damage = None
        if traj.p_key in list(damage_df.p_key):
            traj.damage = damage_df[damage_df.p_key == traj.p_key]['mean'].values[0]

        elif ('fs' in str(traj.p_key)) or ('*' in str(traj.p_key)):
            traj.damage = 1

LiFT_df = pd.DataFrame()
for part in lbc_LiFT:
    for traj in part.trajectories:
        co_ocurring = [traj_2.mutation for traj_2 in part.trajectories if traj_2.mutation != traj.mutation]

        above_2 = False
        if traj.data.AF.max()>0.02:
            above_2 = True
            
        LiFT_df = LiFT_df.append({'participant_id': part.id,
                                  'mutation_id': traj.mutation,
                                  'protein_id': traj.p_key,
                                  '>2%': str(above_2),
                                  'fitness':traj.fitness,
                                  'fitness_confidence_interval': traj.fitness_quantiles,
                                  'co-ocurring mutations': ', '.join(co_ocurring),
                                  'damage': traj.damage,
                                  'outlier_pvalue': traj.outlier_pvalue
                                 }, ignore_index=True)

In [ ]:
LiFT_df.to_csv(path +'LiFT_summary.csv', index=False)

# Threshold summary

In [6]:

# Import non-synonymous mutations as exported in LiFT.py
with open('../Exports/LBC_non-synonymous_threshold_fitted.dill', 'rb') as infile:
    lbc_threshold = dill.load(infile)

for part in lbc_threshold:
    for traj in part.trajectories:
        traj.outlier_pvalue = None
        if traj.mutation in [mutation.id for mutation in outlier_pvalue_mutation_list]:
            pvalue = [mutation.outlier_pvalue for mutation in outlier_pvalue_mutation_list if mutation.id == traj.mutation][0]
            traj.outlier_pvalue = pvalue

damage_df = damage.damage

# Update damage class attribute for each trajectory in the cohort
for part in lbc_threshold:
    for traj in part.trajectories:
        traj.damage = None
        if traj.p_key in list(damage_df.p_key):
            traj.damage = damage_df[damage_df.p_key == traj.p_key]['mean'].values[0]

        elif ('fs' in str(traj.p_key)) or ('*' in str(traj.p_key)):
            traj.damage = 1

threshold_df = pd.DataFrame()
for part in lbc_threshold:
    for traj in part.trajectories:
        co_ocurring = [traj_2.mutation for traj_2 in part.trajectories if traj_2.mutation != traj.mutation]

        above_2 = False
        if traj.data.AF.max()>0.02:
            above_2 = True
            
        threshold_df = threshold_df.append({'participant_id': part.id,
                                  'mutation_id': traj.mutation,
                                  'protein_id': traj.p_key,
                                  '>2%': str(above_2),
                                  'fitness':traj.fitness,
                                  'fitness_confidence_interval': traj.fitness_quantiles,
                                  'co-ocurring mutations': ', '.join(co_ocurring),
                                  'damage': traj.damage}, ignore_index=True)

In [11]:
threshold_df.to_csv(path +'Threshold_summary.csv', index=False)